In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/AI/dataset/Energy_Power_Weather.xlsx", sheet_name=0)

Mounted at /content/drive


# 1.Tiền xử lý dữ liệu

In [2]:
# df = pd.read_excel("Energy_Power_Weather.xlsx", sheet_name=0)
df = df.drop(labels=["Timestamp", "Date"], axis=1)
df

,kilowatt-hours,TempMin,TempMax,Weather
0,2.165,18,23,Patchy rain possible
1,6.470,20,25,Patchy rain possible
2,9.157,21,24,Patchy rain possible
3,16.269,21,25,Sunny
4,8.447,19,23,Moderate rain at times
...,...,...,...,...
1757,4.514,21,23,Light rain shower
1758,11.096,22,26,Patchy rain possible
1759,4.860,20,25,Light rain shower
1760,13.984,23,25,Light rain shower


In [3]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df["Weather"] = le.fit_transform(df["Weather"])
# print(df["Weather"].unique())

# Tạo các biến giả từ các biến phân loại trong df_x và loại bỏ một cột dummy để tránh đa cộng tuyến
data_dummies = pd.get_dummies(df, drop_first=True)
# Chuyển đổi kiểu dữ liệu của tất cả các giá trị trong DataFrame data_dummies thành float
data_dummies = data_dummies.astype(float)
data_dummies

,kilowatt-hours,TempMin,TempMax,Weather_Heavy rain at times,Weather_Light rain shower,Weather_Mist,Weather_Moderate or heavy rain shower,Weather_Moderate rain at times,Weather_Overcast,Weather_Partly cloudy,Weather_Patchy light drizzle,Weather_Patchy light rain with thunder,Weather_Patchy rain possible,Weather_Sunny,Weather_Thundery outbreaks possible,Weather_Torrential rain shower
0,2.165,18.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,6.470,20.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,9.157,21.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,16.269,21.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,8.447,19.0,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,4.514,21.0,23.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1758,11.096,22.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1759,4.860,20.0,25.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1760,13.984,23.0,25.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2.Tạo dữ liệu

In [8]:
from sklearn.model_selection import train_test_split

X = []
y = []
dataframe = data_dummies.values
for i in range(len(dataframe) - 15):
    X.append(dataframe[i:i+14])
    y.append(dataframe[i+15][0])
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

(1747, 14, 16) (1747,)
(1397, 14, 16)


# 3.Huấn luyện

### 3.1. LSTM

In [9]:
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError


def build_lstm(X_train, y_train):
    n_samples, timesteps, n_feature = X_train.shape
    model = Sequential()
    model.add(LSTM(200, activation="tanh", input_shape=(timesteps, n_feature)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer='adam', metrics=[RootMeanSquaredError()])
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return model

md = build_lstm(X_train, y_train)


In [10]:
print(md.evaluate(X_test, y_test, verbose=0))
print(md.predict(X_test, verbose=0)[:7], y_test[:7])

[28.306087493896484, 5.320346355438232]
[[ 6.9632783]
 [ 7.5892706]
 [ 7.9754615]
 [13.022684 ]
 [ 7.090324 ]
 [12.1211605]
 [12.680638 ]] [17.108  1.076  8.959 19.032 10.294 13.739  6.464]


### 3.2 CNN-LSTM

In [11]:
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import Flatten
from keras.layers import RepeatVector
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError

def build_cnn_lstm(X_train, y_train):
    n_samples, time_step, n_feature = X_train.shape
    md = Sequential()
    md.add(Conv1D(64, kernel_size=3, activation="relu", input_shape=(time_step, n_feature)))
    md.add(Conv1D(128, kernel_size=3, activation="relu"))
    md.add(MaxPool1D(2))
    md.add(Flatten())
    md.add(RepeatVector(time_step))
    md.add(LSTM(200, activation="tanh"))
    md.add(Dense(100, activation='relu'))
    md.add(Dense(1))
    md.compile(loss="mse", optimizer="adam", metrics=[RootMeanSquaredError()])
    md.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return md

cnn_lstm = build_cnn_lstm(X_train, y_train)
print(cnn_lstm.evaluate(X_test, y_test, verbose=0))
print(cnn_lstm.predict(X_test, verbose=0)[:7], y_test[:7])

[26.895904541015625, 5.186126232147217]
[[ 9.785398 ]
 [ 7.1140394]
 [ 8.764096 ]
 [16.676003 ]
 [11.941145 ]
 [11.558456 ]
 [11.734069 ]] [17.108  1.076  8.959 19.032 10.294 13.739  6.464]
